# **lsstypes**: hands-on

Let's explain how types are defined in **lsstypes**.

## **ObservableLeaf**, **ObservableTree**

**ObservableLeaf** represents the most basic data unit. Typically:
- one power spectrum multipole
- pair counts
- one BAO parameter
- one angular power spectrum
- ...

Let's consider an example below.

## **ObervableLeaf**

In [1]:
from pathlib import Path
import numpy as np

from lsstypes import ObservableLeaf

s = np.linspace(0., 200., 51)
mu = np.linspace(-1., 1., 101)
rng = np.random.RandomState(seed=42)
counts = 1. + rng.uniform(size=(s.size, mu.size))
# Specify all data entries, and the name of the coordinate axes
# Optionally, some extra attributes
counts = ObservableLeaf(counts=counts, s=s, mu=mu, coords=['s', 'mu'], attrs=dict(los='x'))

In [2]:
# Let's play with this object

assert np.array_equal(counts.coords('s'), counts.s)
assert np.array_equal(counts.coords('mu'), counts.mu)
assert np.array_equal(counts.value(), counts.counts)

# We can slice it
counts2 = counts.select(s=(0., 180.))
print(f'New shape is {counts2.shape}, old shape is {counts.shape}.')

# Let's say that within the range s = (10., 180.) we want to select s = (0., 150.)
counts2 = counts.at(s=(10., 180.)).select(s=(0., 150.))
print(counts2.s)
# There is a gap between 150 and 180, as expected!
# NOTE: This will become relevant if we want to rebin in a portion of the array

New shape is (46, 101), old shape is (51, 101).
[  0.   4.   8.  12.  16.  20.  24.  28.  32.  36.  40.  44.  48.  52.
  56.  60.  64.  68.  72.  76.  80.  84.  88.  92.  96. 100. 104. 108.
 112. 116. 120. 124. 128. 132. 136. 140. 144. 148. 184. 188. 192. 196.
 200.]


In [3]:
# Now let's save it!
test_dir = Path('_tests')

# As HDF5
fn = test_dir / 'test.h5'
counts.write(fn)
counts2 = ObservableLeaf.read(fn)
assert counts2 == counts

# Let's explore the file!
import h5py

def print_type(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"Group:    {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"Dataset:  {name}: {obj[...] if obj.size < 4 else obj[:4]}")

with h5py.File(fn, "r") as f:
    f.visititems(print_type)

Dataset:  coords_names: [b's' b'mu']
Dataset:  counts: [[1.37454012 1.95071431 1.73199394 1.59865848 1.15601864 1.15599452
  1.05808361 1.86617615 1.60111501 1.70807258 1.02058449 1.96990985
  1.83244264 1.21233911 1.18182497 1.18340451 1.30424224 1.52475643
  1.43194502 1.29122914 1.61185289 1.13949386 1.29214465 1.36636184
  1.45606998 1.78517596 1.19967378 1.51423444 1.59241457 1.04645041
  1.60754485 1.17052412 1.06505159 1.94888554 1.96563203 1.80839735
  1.30461377 1.09767211 1.68423303 1.44015249 1.12203823 1.49517691
  1.03438852 1.9093204  1.25877998 1.66252228 1.31171108 1.52006802
  1.54671028 1.18485446 1.96958463 1.77513282 1.93949894 1.89482735
  1.59789998 1.92187424 1.0884925  1.19598286 1.04522729 1.32533033
  1.38867729 1.27134903 1.82873751 1.35675333 1.28093451 1.54269608
  1.14092422 1.80219698 1.07455064 1.98688694 1.77224477 1.19871568
  1.00552212 1.81546143 1.70685734 1.72900717 1.77127035 1.07404465
  1.35846573 1.11586906 1.86310343 1.62329813 1.33089802 1.06

In [4]:
# We can also save as txt

fn = test_dir / 'test.txt'
counts.write(fn)
counts2 = ObservableLeaf.read(fn)
assert counts2 == counts

In [5]:
!ls -R _tests/test

_tests/test:
attrs.json	  counts.txt  name.txt	values_names.txt
coords_names.txt  mu.txt      s.txt


## **ObervableTree**

An **ObservableTree** is a collection of **ObservableTree** or **ObservableLeaf** instances.

In [6]:
from lsstypes import ObservableTree

labels = ['DD', 'DR', 'RR']
leaves = []
for label in labels:
    counts = 1. + rng.uniform(size=(s.size, mu.size))
    leaves.append(ObservableLeaf(counts=counts, s=s, mu=mu, coords=['s', 'mu'], attrs=dict(los='x')))

tree = ObservableTree(leaves, pairs=labels)
print(tree.labels(keys_only=True))
print(tree.labels())
assert len(tree.value()) == tree.size

# Now let's say we want to select pair DD only
tree2 = tree.at(pairs='DD').select(s=(10., 80.))
s1, s2 = tree2.get(pairs='DD').shape, tree2.get('DR').shape
print(f'New shape is {s}, old shape is {s2}.')
# Collective select
tree2 = tree.select(s=(10., 80.))
assert tree2.get(pairs='DD').shape == tree2.get(pairs='DR').shape

correlation = tree

['pairs']
[{'pairs': 'DD'}, {'pairs': 'DR'}, {'pairs': 'RR'}]
New shape is [  0.   4.   8.  12.  16.  20.  24.  28.  32.  36.  40.  44.  48.  52.
  56.  60.  64.  68.  72.  76.  80.  84.  88.  92.  96. 100. 104. 108.
 112. 116. 120. 124. 128. 132. 136. 140. 144. 148. 152. 156. 160. 164.
 168. 172. 176. 180. 184. 188. 192. 196. 200.], old shape is (51, 101).


In [7]:
# We can build something more complicated!

# Let's use some types defined in lsstypes/types.txt

from lsstypes import Spectrum2Pole, Spectrum2Poles

def get_spectrum(seed=None):
    ells = [0, 2, 4]
    rng = np.random.RandomState(seed=seed)
    poles = []
    for ell in ells:
        k = np.linspace(0., 0.2, 41)
        poles.append(Spectrum2Pole(k=k, num=rng.uniform(size=k.size)))
    return Spectrum2Poles(poles, ells=ells)

spectrum = get_spectrum()

tree = ObservableTree([correlation, spectrum], observables=['correlation', 'spectrum'])
tree.get(observables='correlation', pairs='DD').counts;

In [8]:
# Save as HDF5
fn = test_dir / 'tree.h5'
tree.write(fn)
tree2 = ObservableTree.read(fn)
assert tree2 == tree

# Let's explore the file!
with h5py.File(fn, "r") as f:
    f.visititems(print_type)

Group:    correlation
Group:    correlation/DD
Dataset:  correlation/DD/coords_names: [b's' b'mu']
Dataset:  correlation/DD/counts: [[1.20179495 1.9088265  1.76040007 1.67639837 1.30096632 1.18404477
  1.75645109 1.47415795 1.22578955 1.61662198 1.04033385 1.32617025
  1.46855699 1.14795833 1.98465448 1.20909736 1.13035313 1.20353487
  1.76906113 1.42550566 1.56541594 1.02100916 1.03443131 1.44582555
  1.81665524 1.88523888 1.08670657 1.5383631  1.91322969 1.38797275
  1.82956583 1.13369565 1.93539449 1.75142332 1.94133439 1.67661723
  1.36340393 1.93818117 1.27556678 1.33217252 1.70080842 1.7653163
  1.92940348 1.20480241 1.79799804 1.73854605 1.06379456 1.38722314
  1.28276546 1.30449042 1.98294688 1.64277741 1.71767616 1.97661764
  1.37720178 1.80184017 1.43453096 1.86953367 1.18065754 1.94768618
  1.21877171 1.32622051 1.75619223 1.39375514 1.60821138 1.4445735
  1.74236994 1.22833125 1.05845799 1.2999294  1.47430866 1.16802005
  1.35479729 1.39998302 1.05739184 1.58330725 1.884343

In [9]:
# Save as txt
fn = test_dir / 'tree.txt'
tree.write(fn)
tree2 = ObservableTree.read(fn)
assert tree2 == tree

In [10]:
!ls -R _tests/tree

_tests/tree:
attrs.json   labels_names.txt	name.txt
correlation  labels_values.txt	spectrum

_tests/tree/correlation:
attrs.json  DD	DR  labels_names.txt  labels_values.txt  name.txt  RR

_tests/tree/correlation/DD:
attrs.json	  counts.txt  name.txt	values_names.txt
coords_names.txt  mu.txt      s.txt

_tests/tree/correlation/DR:
attrs.json	  counts.txt  name.txt	values_names.txt
coords_names.txt  mu.txt      s.txt

_tests/tree/correlation/RR:
attrs.json	  counts.txt  name.txt	values_names.txt
coords_names.txt  mu.txt      s.txt

_tests/tree/spectrum:
0  2  4  attrs.json  labels_names.txt  labels_values.txt  name.txt

_tests/tree/spectrum/0:
attrs.json	  k.txt     norm.txt	       num.txt
coords_names.txt  name.txt  num_shotnoise.txt  values_names.txt

_tests/tree/spectrum/2:
attrs.json	  k.txt     norm.txt	       num.txt
coords_names.txt  name.txt  num_shotnoise.txt  values_names.txt

_tests/tree/spectrum/4:
attrs.json	  k.txt     norm.txt	       num.txt
coords_names.txt  name.txt  num